https://docs.ragas.io/en/stable/howtos/applications/singlehop_testset_gen/

In [1]:
import ragas

print(ragas.__version__)

0.2.15


In [2]:
import ragas.metrics

print(dir(ragas.metrics))

['AgentGoalAccuracyWithReference', 'AgentGoalAccuracyWithoutReference', 'AnswerAccuracy', 'AnswerCorrectness', 'AnswerRelevancy', 'AnswerSimilarity', 'AspectCritic', 'BleuScore', 'ContextEntityRecall', 'ContextPrecision', 'ContextRecall', 'ContextRelevance', 'ContextUtilization', 'DataCompyScore', 'DistanceMeasure', 'ExactMatch', 'FactualCorrectness', 'Faithfulness', 'FaithfulnesswithHHEM', 'InstanceRubrics', 'LLMContextPrecisionWithReference', 'LLMContextPrecisionWithoutReference', 'LLMContextRecall', 'LLMSQLEquivalence', 'Metric', 'MetricOutputType', 'MetricType', 'MetricWithEmbeddings', 'MetricWithLLM', 'MultiModalFaithfulness', 'MultiModalRelevance', 'MultiTurnMetric', 'NoiseSensitivity', 'NonLLMContextPrecisionWithReference', 'NonLLMContextRecall', 'NonLLMStringSimilarity', 'ResponseGroundedness', 'ResponseRelevancy', 'RougeScore', 'RubricsScore', 'SemanticSimilarity', 'SimpleCriteriaScore', 'SingleTurnMetric', 'StringPresence', 'SummarizationScore', 'ToolCallAccuracy', 'TopicAdhe

In [3]:
! git clone https://huggingface.co/datasets/explodinggradients/ragas-airline-dataset

fatal: destination path 'ragas-airline-dataset' already exists and is not an empty directory.


In [4]:
from langchain_community.document_loaders import DirectoryLoader

path = "ragas-airline-dataset"
loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider insta

In [5]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(
    OpenAIEmbeddings(model="text-embedding-3-small")
)

In [6]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType


kg = KnowledgeGraph()

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={
                "page_content": doc.page_content,
                "document_metadata": doc.metadata,
            },
        )
    )

kg

KnowledgeGraph(nodes: 9, relationships: 0)

In [7]:
from ragas.testset.transforms import apply_transforms
from ragas.testset.transforms import (
    HeadlinesExtractor,
    HeadlineSplitter,
    KeyphrasesExtractor,
)

headline_extractor = HeadlinesExtractor(llm=generator_llm, max_num=20)
headline_splitter = HeadlineSplitter(max_tokens=1500)
keyphrase_extractor = KeyphrasesExtractor(llm=generator_llm)

transforms = [headline_extractor, headline_splitter, keyphrase_extractor]

apply_transforms(kg, transforms=transforms)

Applying HeadlinesExtractor:   0%|          | 0/9 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/9 [00:00<?, ?it/s]

Applying KeyphrasesExtractor:   0%|          | 0/27 [00:00<?, ?it/s]

Property 'keyphrases' already exists in node '2b0acf'. Skipping!


In [8]:
from ragas.testset.persona import Persona

persona_first_time_flier = Persona(
    name="First Time Flier",
    role_description="Is flying for the first time and may feel anxious. Needs clear guidance on flight procedures, safety protocols, and what to expect throughout the journey.",
)

persona_frequent_flier = Persona(
    name="Frequent Flier",
    role_description="Travels regularly and values efficiency and comfort. Interested in loyalty programs, express services, and a seamless travel experience.",
)

persona_angry_business_flier = Persona(
    name="Angry Business Class Flier",
    role_description="Demands top-tier service and is easily irritated by any delays or issues. Expects immediate resolutions and is quick to express frustration if standards are not met.",
)

personas = [
    persona_first_time_flier,
    persona_frequent_flier,
    persona_angry_business_flier,
]

Query Generation Using Synthesizers

Synthesizers are responsible for converting enriched nodes and personas into queries. They achieve this by selecting a node property (e.g., "entities" or "keyphrases"), pairing it with a persona, style, and query length, and then using a LLM to generate a query-answer pair based on the content of the node.

Two instances of the SingleHopSpecificQuerySynthesizer are used to define the query distribution:

Headlines-Based Synthesizer – Generates queries using extracted document headlines, leading to structured questions that reference specific sections.
Keyphrases-Based Synthesizer – Forms queries around key concepts, generating broader, thematic questions.
Both synthesizers are weighted equally (0.5 each), ensuring a balanced mix of specific and conceptual queries, which ultimately enhances the diversity of the test set.

In [9]:
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)

query_distibution = [
    (
        SingleHopSpecificQuerySynthesizer(llm=generator_llm, property_name="headlines"),
        0.5,
    ),
    (
        SingleHopSpecificQuerySynthesizer(
            llm=generator_llm, property_name="keyphrases"
        ),
        0.5,
    ),
]

In [10]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    knowledge_graph=kg,
    persona_list=personas,
)

In [11]:
testset = generator.generate(testset_size=10, query_distribution=query_distibution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,"What to do if my baggage is Delayed, Lost, or ...",[Baggage Policies\n\nThis section provides a d...,"If your baggage is delayed, lost, or damaged, ...",single_hop_specifc_query_synthesizer
1,What to do if payment fail when booking?,[Booking Tickets\n\nOnline Booking\n\nBooking ...,"If your payment does not go through, follow th...",single_hop_specifc_query_synthesizer
2,Can you explain the process involved in Step 3...,[Flight Cancellations\n\nFlight cancellations ...,"In Step 3: Refund Processing, refunds will be ...",single_hop_specifc_query_synthesizer
3,What are the potential issues and resolutions ...,[Flight Delays\n\nFlight delays can be caused ...,Potential issues and resolutions for flight de...,single_hop_specifc_query_synthesizer
4,What can first-time fliers expect regarding me...,[In-Flight Services\n\nRagas Airlines provides...,"On Ragas Airlines, first-time fliers can expec...",single_hop_specifc_query_synthesizer
5,"As a first-time flier, what should I do if my ...",[Baggage Policies This section provides a deta...,"If your baggage is delayed, follow these steps...",single_hop_specifc_query_synthesizer
6,What should I do if I encounter baggage issues...,[Potential Issues and Resolutions for Baggage ...,"If you encounter baggage issues, it is crucial...",single_hop_specifc_query_synthesizer
7,How I do booking tickets for my first flight?,[Booking Tickets],Booking tickets is the first step for your fli...,single_hop_specifc_query_synthesizer
8,What steps should I follow to browse available...,[Online Booking Booking a ticket online throug...,To browse available flights on Ragas Airlines'...,single_hop_specifc_query_synthesizer
9,What should I do if I haven't received my conf...,[Booking via Customer Service If you prefer bo...,If you have not received an email confirmation...,single_hop_specifc_query_synthesizer
